In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0

In [3]:
Q.<alpha> = NumberField(xx^2+3)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [5]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [6]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [7]:
line = Line([y, z])

In [8]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [9]:
eqn_SE2 = P(eqn_SE1.subs({d:(c*c+e*f)/c}) )
sing_cubics_SE2 = P(sing_cubics_SE1.subs({d:(c*c+e*f)/c}))

In [10]:
SE2 = Cubic(eqn_SE2, line, sing_cubics_SE2.factor())

In [11]:
SE2.eckardt_points

[(0, 0, 0, 1), (0, 0, 1, 0)]

In [12]:
%time adm_SE2 = SE2.find_admissible_projectivities()

CPU times: user 1.1 s, sys: 121 ms, total: 1.22 s
Wall time: 3.29 s


In [13]:
len(adm_SE2)

96

In [14]:
simm_SE2 = SE2.find_simmetries(adm_SE2)

In [15]:
len(simm_SE2)

4

In [21]:
def find_conditions_for_subfamilies(cubic, sing_cubics, projectivities, simmetries):
    mon = ((x+y+z+t)^3).monomials()
    conditions = []
    sing_cubics_factored = sing_cubics.factor()
    Eck = lcm([el.conditions for el in cubic.tritangent_planes if el.conditions != 0])
    g_proj = []
    for M in [proj for proj in projectivities if proj not in simmetries]:
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), sing_cubics_factored)    
        minor = matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)
        minor = [remove_sing_factors(el, sing_cubics_factored) for el in minor if el !=0]
        prim_deco = cubic.P.ideal(minor).radical().primary_decomposition()
        for ideale in prim_deco:
            gener = tuple([remove_sing_factors(gen, sing_cubics_factored) for gen in ideale.gens()])
            if 1 not in gener:
                flag = True
                for cond in gener:
                    if cond.divides(Eck):
                        flag = False
                        break
                if flag is True:
                    conditions.append(gener) 
    return list(set(conditions))

def remove_conditions_for_eckardt_points_and_sing(cubic, sing_cubics, conditions):
    vrs = cubic.eqn.variables()[4:]  
    Eck = lcm([el.conditions for el in cubic.tritangent_planes if el.conditions != 0])
    true_cond = []
    for cond in conditions:
        sol = solve_linear_system(cond, vrs[0:-1], [vrs[-1]])
        sost = {vrs[i]:sol[i] for i in range(len(vrs))}
        if sing_cubics.subs(sost) == 0:
            continue
        if Eck.subs({vrs[i]:sol[i] for i in range(len(vrs))}) != 0:
            true_cond.append(cond)
    return true_cond

In [22]:
conditions = find_conditions_for_subfamilies(SE2, sing_cubics_SE2, adm_SE2, simm_SE2)
conditions = remove_conditions_for_eckardt_points_and_sing(SE2, sing_cubics_SE2, conditions)
conditions

[(e - 3*f, c + (-alpha)*f), (3*e - f, 3*c + (alpha)*f), (e - 3*f, c + (alpha)*f), (3*e - f, 3*c + (-alpha)*f)]


[]